In [1]:
import os
os.chdir('../')
os.getcwd()

'/media/nazirnayal/DATA/projects/attention_long_tail/paying-attention-to-the-long-tail'

In [2]:
%load_ext autoreload
%autoreload 2

In [21]:
import torch
import yaml
from collections import namedtuple
from models.dpt.models import DPTSegmentationModel
from torchsummary import summary

In [8]:
def read_config(path):
    with open(path, "r") as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
    return config

In [47]:
config = read_config('configs/segmentation/dpt_config.yaml')
config = namedtuple('config', config.keys())(*config.values())

In [48]:
model = DPTSegmentationModel(config)

In [51]:
N = 5
H = 384
W = 384
C = 3

In [52]:
x = torch.randn((N, C, H, W))

In [53]:
out = model(x)

In [38]:
out.shape

torch.Size([5, 12, 224, 224])

In [39]:
summary(model, x)

Layer (type:depth-idx)                        Output Shape              Param #
├─Module: 1                                   []                        --
|    └─VisionTransformer: 2                   []                        --
|    |    └─Dropout: 3-1                      [-1, 197, 768]            --
|    |    └─LayerNorm: 3-2                    [-1, 197, 768]            1,536
|    └─Sequential: 2                          []                        --
|    |    └─ProjectReadout: 3-3               [-1, 196, 768]            1,180,416
|    |    └─Transpose: 3-4                    [-1, 768, 196]            --
|    └─Sequential: 2                          []                        --
|    |    └─ProjectReadout: 3-5               [-1, 196, 768]            1,180,416
|    |    └─Transpose: 3-6                    [-1, 768, 196]            --
|    └─Sequential: 2                          []                        --
|    |    └─ProjectReadout: 3-7               [-1, 196, 768]            1,180,

Layer (type:depth-idx)                        Output Shape              Param #
├─Module: 1                                   []                        --
|    └─VisionTransformer: 2                   []                        --
|    |    └─Dropout: 3-1                      [-1, 197, 768]            --
|    |    └─LayerNorm: 3-2                    [-1, 197, 768]            1,536
|    └─Sequential: 2                          []                        --
|    |    └─ProjectReadout: 3-3               [-1, 196, 768]            1,180,416
|    |    └─Transpose: 3-4                    [-1, 768, 196]            --
|    └─Sequential: 2                          []                        --
|    |    └─ProjectReadout: 3-5               [-1, 196, 768]            1,180,416
|    |    └─Transpose: 3-6                    [-1, 768, 196]            --
|    └─Sequential: 2                          []                        --
|    |    └─ProjectReadout: 3-7               [-1, 196, 768]            1,180,

In [32]:
model.pretrained.model

VisionTransformer(
  (patch_embed): HybridEmbed(
    (backbone): ResNetV2(
      (stem): Sequential(
        (conv): StdConv2dSame(3, 64, kernel_size=(7, 7), stride=(2, 2), bias=False)
        (norm): GroupNormAct(
          32, 64, eps=1e-05, affine=True
          (act): ReLU(inplace=True)
        )
        (pool): MaxPool2dSame(kernel_size=(3, 3), stride=(2, 2), padding=(0, 0), dilation=(1, 1), ceil_mode=False)
      )
      (stages): Sequential(
        (0): ResNetStage(
          (blocks): Sequential(
            (0): Bottleneck(
              (downsample): DownsampleConv(
                (conv): StdConv2dSame(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (norm): GroupNormAct(
                  32, 256, eps=1e-05, affine=True
                  (act): Identity()
                )
              )
              (conv1): StdConv2dSame(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (norm1): GroupNormAct(
                32, 64, eps=1e-05, 

In [40]:
checkpoint = torch.load('weights/segmentation/dpt_hybrid-ade20k-53898607.pt')

model.load_state_dict(checkpoint)